In [1]:
import pandas as pd
import time
import asyncio
import duckdb

import warnings
warnings.filterwarnings('ignore')

In [11]:
def extractPrincipal(query: str) -> pd.DataFrame:
    duckdb.install_extension('sqlite')
    cnx = duckdb.connect("D:/receita_federal_coleta_tratamento/Coleta_CNPJ_RF/dados_descompactados/cnpj.db")
    query_txt = f"{query}"
    df = pd.read_sql_query(query_txt, cnx)
    print(df.shape)
    df.to_csv("empresas.csv", sep=",")

In [3]:
query = """
SELECT * 
FROM socios 
WHERE nome_socio IN (
    'KATIA MARIA BEZERRA SILVA', 
    'RICARDO ELIAS RESTUM ANTONIO FILHO', 
    'ROGERIO MARCHINI SANTOS', 
    'SURYA GUEDES MENDONCA', 
    'OSVALDO ANTUNES CRUZ JUNIOR');
    """

In [4]:
# extrai os dados do cnpj 
# busca o cnpj em estabelecimentos e faz join com empresas pelo cnpj_basico

In [5]:
df_socios = extractPrincipal(query)

(6859, 12)


In [8]:
df_socios = pd.read_csv("nome_socios.csv")

In [9]:
df_socios.head(2)

,Unnamed: 0,cnpj,cnpj_basico,identificador_de_socio,nome_socio,cnpj_cpf_socio,qualificacao_socio,data_entrada_sociedade,pais,representante_legal,nome_representante,qualificacao_representante_legal,faixa_etaria
0,0,51069166000103,51069166,2,KATIA MARIA BEZERRA SILVA,***821354**,5,20230429,NaN,***000000**,NaN,0,7
1,1,51097535000163,51097535,2,RICARDO ELIAS RESTUM ANTONIO FILHO,***364817**,5,20230501,NaN,***000000**,NaN,0,3


In [10]:
# Criar uma lista de todos os CNPJ do df_socios sem repetí-los para criar outra query com eles
cnpj = df_socios['cnpj'].unique()

In [15]:
query = f"""
SELECT *
FROM estabelecimento
INNER JOIN empresas ON estabelecimento.cnpj_basico = empresas.cnpj_basico
WHERE cnpj IN {tuple(cnpj)}
"""

In [16]:
query

'\nSELECT *\nFROM estabelecimento\nINNER JOIN empresas ON estabelecimento.cnpj_basico = empresas.cnpj_basico\nWHERE cnpj IN (51069166000103, 51097535000163, 51106949000101, 51107570000116, 51108409000167, 51108465000100, 51114120000150, 51115045000142, 51115454000149, 51117294000177, 51177443000193, 51189802000122, 51296216000187, 51296228000101, 51324208000105, 51324239000158, 51375925000158, 51381902000156, 51485214000136, 51486185000127, 51632684000185, 51692933000128, 51712880000160, 51722614000118, 51724887000100, 51742055000108, 51747572000170, 51747739000100, 51839976000193, 51847231000176, 51852856000126, 51856007000140, 51869146000109, 51916710000105, 52109885000165, 47694738000160, 52484527000132, 52505972000131, 53026018000129, 53034712000198, 53038658000159, 53633617000100, 36156964000170, 36701526000146, 36711847000121, 36725925000147, 36754609000101, 36874426000111, 36896733000101, 36905236000114, 36906504000112, 36919450000120, 36964976000121, 36964986000167, 37054586000

In [17]:
extractPrincipal(query)

(4561, 38)


In [18]:
df_empresas = pd.read_csv("empresas.csv")

In [19]:
df_empresas.columns

Index(['Unnamed: 0', 'cnpj_basico', 'cnpj_ordem', 'cnpj_dv', 'matriz_filial',
       'nome_fantasia', 'situacao_cadastral', 'data_situacao_cadastral',
       'motivo_situacao_cadastral', 'nome_cidade_exterior', 'pais',
       'data_inicio_atividades', 'cnae_fiscal', 'cnae_fiscal_secundaria',
       'tipo_logradouro', 'logradouro', 'numero', 'complemento', 'bairro',
       'cep', 'uf', 'municipio', 'ddd1', 'telefone1', 'ddd2', 'telefone2',
       'ddd_fax', 'fax', 'correio_eletronico', 'situacao_especial',
       'data_situacao_especial', 'cnpj', 'cnpj_basico.1', 'razao_social',
       'natureza_juridica', 'qualificacao_responsavel', 'porte_empresa',
       'ente_federativo_responsavel', 'capital_social'],
      dtype='object')

In [20]:
# Cria um dataframe com os cnpj, nome_socio, cnpj_cpf_socio, qualificacao_socio do df_socios e 'razao_social', 'uf', 'municipio', 'porte_empresa', 'matriz_filial' da df_empresas
df_final = pd.merge(
    df_socios[['cnpj', 'nome_socio', 'cnpj_cpf_socio', 'qualificacao_socio']], 
    df_empresas[['cnpj', 'razao_social', 'uf', 'municipio', 'porte_empresa', 'matriz_filial']], 
    how='inner', 
    on='cnpj')


In [ ]:
df_final.to_csv("network_socio_empresas.csv", sep=",")